# Tutorial 7: Coordination Protocols

**Goal:** Understand HERON's protocol system and create custom coordination strategies.

**Time:** ~15 minutes

---

## Why Protocols Matter

Protocols define **how coordinators distribute actions to subordinates** and **how peers share information**. HERON's protocol design lets you:

- **Swap coordination strategies** without changing agent code
- **Compose** communication and action protocols independently
- **Compare strategies** for research (e.g., centralized vs. distributed)

### Protocol Architecture

Every protocol is composed of two independent pieces:

```
Protocol
├── CommunicationProtocol  — WHAT to communicate (messages)
└── ActionProtocol          — HOW to coordinate actions
```

### Built-in Protocols

| Protocol | Communication | Action | Use Case |
|----------|--------------|--------|---------|
| `VerticalProtocol` | `NoCommunication` | `VectorDecompositionActionProtocol` | Hierarchical control |
| `HorizontalProtocol` | `StateShareCommunicationProtocol` | `NoActionCoordination` | Peer-to-peer systems |
| `NoProtocol` | `NoCommunication` | `NoActionCoordination` | Independent agents |

In [1]:
import numpy as np
from typing import Any, Dict, Optional

from heron.protocols.base import (
    Protocol,
    CommunicationProtocol,
    ActionProtocol,
    NoCommunication,
    NoActionCoordination,
    NoProtocol,
)
from heron.protocols.vertical import VerticalProtocol, VectorDecompositionActionProtocol
from heron.protocols.horizontal import HorizontalProtocol, StateShareCommunicationProtocol
from heron.utils.typing import AgentID

print("Imports ready!")

Imports ready!


## Step 1: The Protocol Interface

A `Protocol` composes a `CommunicationProtocol` and an `ActionProtocol`. Its main entry point is `coordinate()`:

```python
messages, actions = protocol.coordinate(
    coordinator_state=...,       # State of the coordinating agent
    coordinator_action=...,      # Action from coordinator's policy
    info_for_subordinates=...,   # Dict of subordinate observations
    context=...,                 # Optional additional context
)
# messages: Dict[AgentID, Dict]      — coordination messages per agent
# actions:  Dict[AgentID, Any]       — distributed actions per subordinate
```

Internally, `coordinate()` runs two steps:
1. `communication_protocol.compute_coordination_messages()` — computes messages
2. `action_protocol.compute_action_coordination()` — distributes actions

In [2]:
# Inspect the Protocol base class
import inspect
print(inspect.getsource(Protocol.coordinate))

    def coordinate(
        self,
        coordinator_state: Any,
        coordinator_action: Optional[Any] = None,
        info_for_subordinates: Optional[Dict[AgentID, Any]] = None,
        context: Optional[Dict[str, Any]] = None
    ) -> Tuple[Dict[AgentID, Dict[str, Any]], Dict[AgentID, Any]]:
        """Execute full coordination cycle.

        This is the main entry point that orchestrates:
        1. Communication: Compute and deliver messages
        2. Action: Compute and apply coordinated actions

        Args:
            coordinator_state: State of coordinating agent
            info_for_subordinates: Information for subordinate agents
            coordinator_action: Action from coordinator policy (if any)
            context: Additional context (subordinates dict, timestamp, etc.)

        Returns:
            Tuple of (messages, actions)
        """
        context = context or {}

        # Step 1: Communication coordination
        messages = self.communication_protoco

## Step 2: VerticalProtocol (Hierarchical Control)

The most common protocol. A coordinator computes a **joint action vector** and the protocol splits it across subordinates by their action dimensions.

```
Coordinator action: [0.5, 0.3, -0.2, 0.8]   (4 values)
    │
    ├── Subordinate A (action_dim=2) → [0.5, 0.3]
    └── Subordinate B (action_dim=2) → [-0.2, 0.8]
```

Default components:
- **Communication:** `NoCommunication` (subordinates only receive actions)
- **Action:** `VectorDecompositionActionProtocol` (splits joint action vector)

In [3]:
# VerticalProtocol in action
vertical = VerticalProtocol()

print(f"Communication: {type(vertical.communication_protocol).__name__}")
print(f"Action:        {type(vertical.action_protocol).__name__}")
print(f"No-op?         {vertical.no_op()}")
print(f"No action?     {vertical.no_action()}")
print(f"No comms?      {vertical.no_communication()}")

Communication: NoCommunication
Action:        VectorDecompositionActionProtocol
No-op?         False
No action?     False
No comms?      True


In [4]:
# Simulate vector decomposition
# In real usage, register_subordinates() is called automatically during agent init.
# Here we call the action protocol directly.

action_protocol = VectorDecompositionActionProtocol()

# Coordinator outputs joint action [0.5, 0.3, -0.2]
joint_action = np.array([0.5, 0.3, -0.2])

# Subordinate observations (used as keys for distribution)
sub_infos = {"gen_1": "obs_gen", "ess_1": "obs_ess", "load_1": "obs_load"}

# Without registered dimensions, each subordinate gets the full action
actions = action_protocol.compute_action_coordination(
    coordinator_action=joint_action,
    info_for_subordinates=sub_infos,
)
print("Without registered dims (broadcast):")
for sub_id, action in actions.items():
    print(f"  {sub_id}: {action}")

Without registered dims (broadcast):
  gen_1: [ 0.5  0.3 -0.2]
  ess_1: [ 0.5  0.3 -0.2]
  load_1: [ 0.5  0.3 -0.2]


In practice, `register_subordinates()` is called automatically when the coordinator builds its agent hierarchy. The protocol reads each subordinate's `action.dim_c + action.dim_d` to know how many values to slice.

**Usage with agents** (from Tutorial 2):

```python
from powergrid.agents import PowerGridAgent, Generator, ESS
from heron.protocols.vertical import VerticalProtocol

devices = {
    'gen_1': Generator(agent_id='gen_1', ...),
    'ess_1': ESS(agent_id='ess_1', ...),
}

mg = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=VerticalProtocol(),  # <-- inject protocol
)
```

## Step 3: HorizontalProtocol (Peer-to-Peer)

For peer-to-peer coordination where agents share state with neighbors but make **independent decisions**.

```
Agent A ←──state──→ Agent B ←──state──→ Agent C
  │                    │                    │
  └── act alone        └── act alone        └── act alone
```

Default components:
- **Communication:** `StateShareCommunicationProtocol` (shares state with neighbors)
- **Action:** `NoActionCoordination` (agents decide independently)

The environment owns horizontal protocols (it has the global view needed to route messages).

In [5]:
# HorizontalProtocol with custom topology
horizontal = HorizontalProtocol(
    state_fields=["power", "soc"],  # Only share these fields
    topology={
        "mg_1": ["mg_2"],           # mg_1 sees mg_2
        "mg_2": ["mg_1", "mg_3"],   # mg_2 sees mg_1 and mg_3
        "mg_3": ["mg_2"],           # mg_3 sees mg_2
    }
)

print(f"Communication: {type(horizontal.communication_protocol).__name__}")
print(f"Action:        {type(horizontal.action_protocol).__name__}")

Communication: StateShareCommunicationProtocol
Action:        NoActionCoordination


In [6]:
# Simulate state sharing
# Each agent has a state dict (simplified for demo)
agent_states = {
    "mg_1": {"power": 2.5, "soc": 0.8, "cost": 100},
    "mg_2": {"power": 1.0, "soc": 0.3, "cost": 50},
    "mg_3": {"power": 3.0, "soc": 0.6, "cost": 150},
}

# Note: StateShareCommunicationProtocol filters by state_fields when states
# have a .local attribute (real agent states). With plain dicts, it shares as-is.
messages = horizontal.communication_protocol.compute_coordination_messages(
    sender_state=None,  # Not used for horizontal protocols
    receiver_states=agent_states,
)

for agent_id, msg in messages.items():
    print(f"{agent_id} receives:")
    print(f"  type: {msg['type']}")
    print(f"  neighbors: {msg['neighbors']}")
    print()

mg_1 receives:
  type: neighbor_states
  neighbors: {'mg_2': {'power': 1.0, 'soc': 0.3}}

mg_2 receives:
  type: neighbor_states
  neighbors: {'mg_1': {'power': 2.5, 'soc': 0.8}, 'mg_3': {'power': 3.0, 'soc': 0.6}}

mg_3 receives:
  type: neighbor_states
  neighbors: {'mg_2': {'power': 1.0, 'soc': 0.3}}



## Step 4: Writing a Custom ActionProtocol

To create a custom action distribution strategy, subclass `ActionProtocol` and implement `compute_action_coordination()`.

**Example:** A proportional protocol that distributes a coordinator's scalar action to subordinates based on capacity weights.

In [7]:
class ProportionalActionProtocol(ActionProtocol):
    """Distributes coordinator action proportionally based on weights.

    Instead of slicing a joint vector (like VectorDecomposition), this protocol
    interprets the coordinator action as a single scalar setpoint and distributes
    it proportionally to subordinates based on configured weights.

    Example:
        Coordinator action = 0.6 (total power setpoint)
        Weights: gen_1=3.0, gen_2=1.0 (3:1 ratio)
        -> gen_1 gets 0.45, gen_2 gets 0.15
    """

    def __init__(self, weights: Optional[Dict[AgentID, float]] = None):
        self.weights = weights or {}

    def compute_action_coordination(
        self,
        coordinator_action: Optional[Any],
        info_for_subordinates: Optional[Dict[AgentID, Any]] = None,
        coordination_messages: Optional[Dict[AgentID, Dict[str, Any]]] = None,
        context: Optional[Dict[str, Any]] = None,
    ) -> Dict[AgentID, Any]:
        if coordinator_action is None or info_for_subordinates is None:
            return {sub_id: None for sub_id in (info_for_subordinates or {})}

        # Extract scalar action value
        if hasattr(coordinator_action, 'c'):
            total = float(coordinator_action.c[0])
        elif isinstance(coordinator_action, np.ndarray):
            total = float(coordinator_action[0])
        else:
            total = float(coordinator_action)

        # Compute normalized weights
        sub_ids = list(info_for_subordinates.keys())
        if not self.weights:
            norm_weights = {sid: 1.0 / len(sub_ids) for sid in sub_ids}
        else:
            total_w = sum(self.weights.get(sid, 0.0) for sid in sub_ids)
            if total_w == 0:
                norm_weights = {sid: 1.0 / len(sub_ids) for sid in sub_ids}
            else:
                norm_weights = {
                    sid: self.weights.get(sid, 0.0) / total_w for sid in sub_ids
                }

        return {
            sid: np.array([total * norm_weights[sid]])
            for sid in sub_ids
        }


print("ProportionalActionProtocol defined!")

ProportionalActionProtocol defined!


In [8]:
# Test the custom action protocol
prop_action = ProportionalActionProtocol(
    weights={"large_gen": 3.0, "small_gen": 1.0}
)

actions = prop_action.compute_action_coordination(
    coordinator_action=np.array([0.8]),
    info_for_subordinates={"large_gen": "obs_1", "small_gen": "obs_2"},
)

print("ProportionalActionProtocol (total=0.8, weights 3:1):")
for sub_id, action in actions.items():
    print(f"  {sub_id}: {action[0]:.3f}")

ProportionalActionProtocol (total=0.8, weights 3:1):
  large_gen: 0.600
  small_gen: 0.200


## Step 5: Wrapping in a Protocol

To use a custom `ActionProtocol` with agents, wrap it in a `Protocol` subclass.

The wrapper composes your custom action protocol with a communication protocol (typically `NoCommunication` for vertical coordination).

In [9]:
class ProportionalProtocol(Protocol):
    """Vertical protocol with proportional action distribution."""

    def __init__(self, weights: Optional[Dict[AgentID, float]] = None):
        super().__init__(
            communication_protocol=NoCommunication(),
            action_protocol=ProportionalActionProtocol(weights),
        )


# Use it like any other protocol
proto = ProportionalProtocol(weights={"gen_1": 3.0, "ess_1": 1.0})

messages, actions = proto.coordinate(
    coordinator_state=None,
    coordinator_action=np.array([0.6]),
    info_for_subordinates={"gen_1": "obs_1", "ess_1": "obs_2"},
)

print("ProportionalProtocol.coordinate():")
print(f"  Messages: {messages}")
print(f"  Actions:")
for sub_id, action in actions.items():
    print(f"    {sub_id}: {action[0]:.3f}")

ProportionalProtocol.coordinate():
  Messages: {'gen_1': {}, 'ess_1': {}}
  Actions:
    gen_1: 0.450
    ess_1: 0.150


## Step 6: Writing a Custom CommunicationProtocol

For peer-to-peer systems, you can customize **what information agents share**.

**Example:** A price signal protocol where a coordinator broadcasts a price and agents receive it as a message (rather than a direct action).

In [10]:
from typing import Set


class PriceSignalCommunicationProtocol(CommunicationProtocol):
    """Broadcasts a price signal to all receivers.

    The sender (coordinator) computes a price signal from its state.
    All receivers get the same price message, which they can use
    to make local decisions.
    """

    def __init__(self, base_price: float = 50.0):
        self.neighbors: Set[AgentID] = set()
        self.base_price = base_price

    def compute_coordination_messages(
        self,
        sender_state: Any,
        receiver_infos: Dict[AgentID, Any],
        context: Optional[Dict[str, Any]] = None,
    ) -> Dict[AgentID, Dict[str, Any]]:
        price = self.base_price
        if sender_state is not None and hasattr(sender_state, 'features'):
            # In practice, read from a feature (e.g., market clearing price)
            pass

        return {
            receiver_id: {"type": "price_signal", "price": price}
            for receiver_id in receiver_infos
        }


# Combine with NoActionCoordination for a "price broadcast" protocol
class PriceBroadcastProtocol(Protocol):
    """Broadcasts price signals without coordinating actions.

    Agents receive price info and decide their actions independently.
    """

    def __init__(self, base_price: float = 50.0):
        super().__init__(
            communication_protocol=PriceSignalCommunicationProtocol(base_price),
            action_protocol=NoActionCoordination(),
        )


# Test
price_proto = PriceBroadcastProtocol(base_price=60.0)

messages, actions = price_proto.coordinate(
    coordinator_state=None,
    info_for_subordinates={"gen_1": "obs_1", "ess_1": "obs_2"},
)

print("PriceBroadcastProtocol.coordinate():")
print(f"  Messages:")
for sub_id, msg in messages.items():
    print(f"    {sub_id}: {msg}")
print(f"  Actions:")
for sub_id, action in actions.items():
    print(f"    {sub_id}: {action}  (None = agent decides locally)")

PriceBroadcastProtocol.coordinate():
  Messages:
    gen_1: {'type': 'price_signal', 'price': 60.0}
    ess_1: {'type': 'price_signal', 'price': 60.0}
  Actions:
    gen_1: None  (None = agent decides locally)
    ess_1: None  (None = agent decides locally)


## Step 7: Composing Communication + Action

The real power of the two-piece design: combine a custom `CommunicationProtocol` with a custom `ActionProtocol` to build hybrid strategies.

In [11]:
class PriceGuidedProtocol(Protocol):
    """Combines price broadcast with proportional action distribution.

    Subordinates receive both:
    - A price signal (for local optimization context)
    - A proportional share of the coordinator's action
    """

    def __init__(
        self,
        base_price: float = 50.0,
        weights: Optional[Dict[AgentID, float]] = None,
    ):
        super().__init__(
            communication_protocol=PriceSignalCommunicationProtocol(base_price),
            action_protocol=ProportionalActionProtocol(weights),
        )


# Test the hybrid protocol
hybrid = PriceGuidedProtocol(
    base_price=55.0,
    weights={"gen_1": 2.0, "ess_1": 1.0},
)

messages, actions = hybrid.coordinate(
    coordinator_state=None,
    coordinator_action=np.array([0.9]),
    info_for_subordinates={"gen_1": "obs_1", "ess_1": "obs_2"},
)

print("PriceGuidedProtocol.coordinate():")
print(f"  Messages:")
for sub_id, msg in messages.items():
    print(f"    {sub_id}: {msg}")
print(f"  Actions:")
for sub_id, action in actions.items():
    print(f"    {sub_id}: {action[0]:.3f}")

PriceGuidedProtocol.coordinate():
  Messages:
    gen_1: {'type': 'price_signal', 'price': 55.0}
    ess_1: {'type': 'price_signal', 'price': 55.0}
  Actions:
    gen_1: 0.600
    ess_1: 0.300


## Step 8: Plugging Protocols into Agents

The key HERON pattern: **change coordination without touching agent code**.

```python
from powergrid.agents import PowerGridAgent, Generator, ESS
from heron.protocols.vertical import VerticalProtocol

devices = {
    'gen_1': Generator(agent_id='gen_1', bus='bus_1', p_max_MW=10.0),
    'ess_1': ESS(agent_id='ess_1', bus='bus_1', capacity_MWh=5.0),
}

# Strategy 1: Direct vector decomposition (default)
mg_v1 = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=VerticalProtocol(),
)

# Strategy 2: Proportional distribution by capacity
mg_v2 = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=ProportionalProtocol(weights={'gen_1': 3.0, 'ess_1': 1.0}),
)

# Strategy 3: Price-guided hybrid
mg_v3 = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=PriceGuidedProtocol(base_price=50.0, weights={'gen_1': 3.0, 'ess_1': 1.0}),
)
```

All three use the **same agent class** and **same training loop**. Only the protocol changes.

## Step 9: Protocol in the Training Loop

During CTDE training, the coordinator's action is distributed via `protocol.coordinate()`:

```python
# Inside training loop (see tests/integration/test_action_passing.py for full example)
coordinator_agent = env.registered_agents["coordinator"]

# Coordinator policy computes joint action
coordinator_action = policy.forward(aggregated_obs)

# Protocol distributes to subordinates
_, distributed_actions = coordinator_agent.protocol.coordinate(
    coordinator_state=coordinator_agent.state,
    coordinator_action=coordinator_action,
    info_for_subordinates={sub_id: obs[sub_id] for sub_id in sub_ids},
)

# Pass distributed actions to env.step()
obs, rewards, terminated, _, info = env.step(distributed_actions)
```

## Summary

### Protocol Architecture

```python
from heron.protocols.base import Protocol, CommunicationProtocol, ActionProtocol

# Protocols compose two independent pieces:
Protocol(
    communication_protocol=MyCommunication(),  # WHAT to share
    action_protocol=MyActionCoord(),            # HOW to distribute actions
)
```

### Custom ActionProtocol

```python
class MyActionProtocol(ActionProtocol):
    def compute_action_coordination(
        self, coordinator_action, info_for_subordinates,
        coordination_messages=None, context=None,
    ) -> Dict[AgentID, Any]:
        return {sub_id: ... for sub_id in info_for_subordinates}
```

### Custom CommunicationProtocol

```python
class MyCommunication(CommunicationProtocol):
    def __init__(self):
        self.neighbors: Set[AgentID] = set()

    def compute_coordination_messages(
        self, sender_state, receiver_infos, context=None,
    ) -> Dict[AgentID, Dict[str, Any]]:
        return {receiver_id: {...} for receiver_id in receiver_infos}
```

### Built-in Protocols

| Class | Type | Purpose |
|-------|------|---------|
| `VerticalProtocol` | Hierarchical | Vector decomposition (coordinator splits joint action) |
| `HorizontalProtocol` | Peer-to-peer | State sharing (agents share info, act independently) |
| `NoProtocol` | None | Fully independent agents |

### Pluggable Design

```python
# Same agent, different coordination — only the protocol changes
mg = PowerGridAgent(agent_id='mg_1', subordinates=devs, protocol=MyProtocol())
```

---

**See also:** `tests/integration/test_action_passing.py` for a full end-to-end example with training and event-driven execution.